In [77]:
import numpy as np
import pandas as pd
import itertools as it
import math as mt

# Ejercicio II.1

### Se ha clasificado el peso de los huevos, $Y$, de una especie de pez en función del peso de la madre, $X$, como refleja la tabla adjunta, y se pide estimar una serie de características de la distribución.

$X$/$Y$    | [25, 27) | [27, 29) | [29, 31) | [31, 33)
---------- | -------- | -------- | -------- | --------
**[500, 550)** | 15       | 11       | 18       | 0
**[550, 600)** | 12       | 14       | 0        | 12
**[600, 650)** | 0        | 3        | 7        | 18

En primer lugar, se codifica la tabla bidimensional del enunciado; se definen sendas *listas* con los intervalos en que se han agrupado ambas variables, $X$ e $Y$, y se indican las **frecuencias absolutas** $n_{ij}$ de cada **par de valores** $(x_{i}, y_{j})$:

In [78]:
valores = [15, 11, 18, 0, 12, 14, 0, 12, 0, 3, 7, 18]

Y = ["[25, 27)", "[27, 29)", "[29, 31)", "[31, 33)"] * 3
Y = np.repeat(Y, valores, axis = 0)

X = ["[500, 550)"] * 4 + ["[550, 600)"] * 4 + ["[600, 650)"] * 4 
X = np.repeat(X, valores, axis = 0)

tabla2D = pd.crosstab(index = X, columns = Y, rownames = ['X'], colnames = ['Y'])
tabla2D

Y,"[25, 27)","[27, 29)","[29, 31)","[31, 33)"
X,,,,
"[500, 550)",15,11,18,0
"[550, 600)",12,14,0,12
"[600, 650)",0,3,7,18


### a) *Distribución del peso del huevo*

Equivale a calcular la **distribución marginal** de $Y$; es decir, sus **frecuencias absolutas marginales** $n_{.j}$ :

In [79]:
n·j = tabla2D.sum()
n·j

Y
[25, 27)    27
[27, 29)    28
[29, 31)    25
[31, 33)    30
dtype: int64

Siendo la distribución marginal de $X$:

In [80]:
ni· = tabla2D.transpose().sum()
ni·

X
[500, 550)    44
[550, 600)    38
[600, 650)    28
dtype: int64

El **número total de observaciones**, $n$ o $n_{..}$:

In [81]:
n = n·j.sum() #ni·.sum() daría el mismo resultado
n

110

En conjunto:

In [82]:
tablaMarg = pd.crosstab(index = X, columns = Y, margins = True)

tablaMarg.index = ["[500, 550)", "[550, 600)", "[600, 650)", "Huevo"]
tablaMarg.columns = ["[25, 27)", "[27, 29)", "[29, 31)", "[31, 33)", "Madre"]

tablaMarg

,"[25, 27)","[27, 29)","[29, 31)","[31, 33)",Madre
"[500, 550)",15,11,18,0,44
"[550, 600)",12,14,0,12,38
"[600, 650)",0,3,7,18,28
Huevo,27,28,25,30,110


### b) *Distribución del peso de la madre cuando el huevo tiene el suyo comprendido entre 25 y 27*

Se trata de la **distribución condicionada** de $X$ a $Y$ cuando ésta toma valores únicamente en el intervalo [25, 27); es decir, las **frecuencias relativas condicionadas** $f_{i|1}$:

In [83]:
fi_1 = tabla2D.transpose().iloc[0]/tablaMarg.iloc[3, 0]

fi_1Red = round(fi_1, 2)
fi_1Red

X
[500, 550)    0.56
[550, 600)    0.44
[600, 650)    0.00
Name: [25, 27), dtype: float64

Si se calcula para todos los intervalos de peso de los huevos, $f_{i|j}$:

In [84]:
fi_j = tablaMarg.div(tablaMarg.loc["Huevo"], axis = 1)

fi_jRed = round(fi_j, 2)
fi_jRed

,"[25, 27)","[27, 29)","[29, 31)","[31, 33)",Madre
"[500, 550)",0.56,0.39,0.72,0.0,0.40
"[550, 600)",0.44,0.50,0.00,0.4,0.35
"[600, 650)",0.00,0.11,0.28,0.6,0.25
Huevo,1.00,1.00,1.00,1.0,1.00


Y si se quiere calcular la distribución del peso de los huevos condicionada al peso de la madre, $f_{j|i}$:

In [85]:
fj_i = tablaMarg.div(tablaMarg["Madre"], axis = 0) #No hay que usar "loc" en este caso

fj_iRed = round(fj_i, 2)
fj_iRed

,"[25, 27)","[27, 29)","[29, 31)","[31, 33)",Madre
"[500, 550)",0.34,0.25,0.41,0.00,1.0
"[550, 600)",0.32,0.37,0.00,0.32,1.0
"[600, 650)",0.00,0.11,0.25,0.64,1.0
Huevo,0.25,0.25,0.23,0.27,1.0


### c) *Media, mediana y moda del peso de los huevos*

En primer lugar, se crea una lista con las **marcas de clase** de $Y$, $y_{j}$:

In [86]:
l1 = 25 #Límite inferior del primer intervalo
l2 = 27 #Límite superior del primer intervalo
a = l2 - l1 #Amplitud de los intervalos
s = 4 #Número de intervalos

y1 = (l1 + l2)/2 #Primera marca de clase

mc = it.count(start = y1, step = a)
yj = [next(mc) for elemento in range(s)]
yj

[26.0, 28.0, 30.0, 32.0]

Y a partir de él, las frecuencias absolutas marginales de $Y$ y el número total de observaciones, se determina la **media** ($\bar{y}$):

In [87]:
media = round(((yj * n·j).sum())/n, 2) 
media

29.05

Para determinar la **mediana** ($Me_{y}$), en primer lugar se determina el intervalo de $Y$ cuya **frecuencia relativa acumulada marginal** ($F_{.j}$) es superior a 0,5:

In [88]:
F·j = np.cumsum(n·j/n)

intervalos = []

for intervalo, frecuencia in F·j.items():
    if frecuencia > 0.5:
        intervalos.append(intervalo)
        
intMed = intervalos[0] #La mediana se encuentra en el inmediatamente superior
intMed

'[29, 31)'

Se toma el extremo inferior de este intervalo:

In [89]:
lMed = int(intMed[1:3])
lMed

29

Se determina la **frecuencia absoluta acumulada marginal** ($N_{.j}$) del intervalo anterior:

In [90]:
for intervalo, frecuencia in F·j.items():
    if frecuencia <= 0.5:
        intervalos.append(intervalo)
        
intAnt = intervalos[-1] #Intervalo inmediatamente inferior al que contiene la mediana
intAnt

N·j = np.cumsum(n·j)

N·jAnt = N·j[intAnt]
N·jAnt

55

Y la frecuencia absoluta marginal del intervalo que contiene la mediana:

In [91]:
n·jMed = n·j[intMed]
n·jMed

25

Finalmente, la mediana se calcula como:

In [92]:
mediana = int(lMed + a * ((n/2 - N·jAnt)/n·jMed))
mediana

29

Con respecto a la **moda**, se puede estimar el **intervalo modal**, aquel con mayor **frecuencia relativa marginal** ($f_{.j}$):

In [93]:
f·j = n·j/n

for intervalo, frecuencia in f·j.items():
    if frecuencia == f·j.max():
        print(intervalo)

[31, 33)


### d) *Nivel de representatividad de la media del peso de la madre cuando el huevo está comprendido entre 25 y 27*

En primer lugar, se determinan las **marcas de clase**, pero en este caso para la variable $X$, el peso de la madre ($x_{i}$):

In [94]:
l1 = 500 #Límite inferior del primer intervalo
l2 = 550 #Límite superior del primer intervalo
a = l2 - l1 #Amplitud de los intervalos
s = 3 #Número de intervalos

x1 = (l1 + l2)/2 #Primera marca de clase

mc = it.count(start = x1, step = a)
xi = [next(mc) for elemento in range(s)]
xi

[525.0, 575.0, 625.0]

Resulta interesante recodificar la tabla de frecuencias bivariada usando las marcas de clase en vez de los intervalos:

In [95]:
tablaMarcas = pd.crosstab(index = X, columns = Y, margins = True)

tablaMarcas.index = xi + ["Huevo"]
tablaMarcas.columns = yj  + ["Madre"]

tablaMarcas

,26.0,28.0,30.0,32.0,Madre
525.0,15,11,18,0,44
575.0,12,14,0,12,38
625.0,0,3,7,18,28
Huevo,27,28,25,30,110


A partir de las marcas de clase y de las **frecuencias relativas condicionadas** $f_{i|1}$, se puede determinar la **media** ($\bar{x}$) de $X$ en relación al primer intervalo de $Y$:

In [96]:
media = sum(xi * fi_1)

mediaRed = round(media, 2)
mediaRed

547.22

Y a partir de la media se puede calcular la **desviación típica** ($S_{x}$):

In [97]:
var = sum(np.power(xi, 2) * fi_1) - media**2

DT = mt.sqrt(var)

DTRed = round(DT, 2)
DTRed

24.85

Con ambas, se calcula el **coeficiente de variación** ($CV$), que es el que indica el nivel de representatividad de la media:

In [98]:
CV = round(DT/media, 3)
CV

0.045

### e) *Independencia de las variables*

Se va a estimar a partir del par $(x_{1}, y_{1})$:

In [99]:
f11 = tabla2D.iloc[0, 0]/n
f1· = ni·.iloc[0]/n
f·1 = n·j.iloc[0]/n

f11 == f1· * f·1

False

Dado que no se verifica la igualdad, las variables no son independientes.

### f) *Dependencia lineal de las variables*

Se va a estimar a través del **coeficiente de correlación de Pearson** ($r$); en primer lugar, se calculan las varianzas de ambas variables:

In [100]:
fi· = ni·/n #Frecuencias relativas marginales de x

mediaX = sum(xi * fi·)

varX = sum(np.power(xi, 2) * fi·) - mediaX**2

varXRed = round(varX, 2)
varXRed

1583.47

In [101]:
f·j = n·j/n #Frecuencias relativas marginales de y

mediaY = sum(yj * f·j)

varY = sum(np.power(yj, 2) * f·j) - mediaY**2

varYRed = round(varY, 2)
varYRed

5.14

En segundo lugar, la **covarianza** ($S_{XY}$):

In [102]:
a = (yj * np.array([xi[0]] * 4) * tabla2D.iloc[0, ])/n  
b = (yj * np.array([xi[1]] * 4) * tabla2D.iloc[1, ])/n 
c = (yj * np.array([xi[2]] * 4) * tabla2D.iloc[2, ])/n
 
covar = (a.sum() + b.sum() + c.sum()) - (mediaX * mediaY)

covarRed = round(covar, 2)
covarRed

44.03

Y, finalmente, el coeficiente:

In [103]:
r = round(covar/mt.sqrt(varX * varY), 2)
r

0.49